In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import mne
from pathlib import Path
import os
path = Path('../data/smell')

In [2]:
subjects = list({s[:9] for s in os.listdir(path/'raw') if s.find('.fif') != -1})

In [3]:
subj = subjects[0]
f_name = f'{subj}_inhale-epo.fif'
eeg = mne.read_epochs(path / 'raw' / f_name)
eeg.resample(125)

Reading ../data/smell/raw/A4SHD_21F_inhale-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    7000.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
120 matching events found
No baseline correction applied
0 projection items activated


<EpochsFIF |  120 events (all good), 0 - 7 sec, baseline off, ~15.3 MB, data loaded,
 'cit_inhale': 30
 'cof_inhale': 30
 'van_inhale': 30
 'wat_inhale': 30>

In [5]:
eeg.event_id

{'cit_inhale': 1, 'cof_inhale': 2, 'van_inhale': 3, 'wat_inhale': 4}

In [33]:
baseline_features = pd.DataFrame()

data = {'means': [], 'stds': [], 'target': []}


for event_id in eeg.event_id:
    for i in range(len(eeg[event_id])):
        powers, freqs = mne.time_frequency.psd_multitaper(eeg[event_id][i], fmin=8, fmax=12, verbose=False)
        data['means'].append({f"{ch}_mean": powers[0, i].mean() for i, ch in enumerate(eeg.ch_names)})
        data['stds'].append({f"{ch}_std": powers[0, i].std() for i, ch in enumerate(eeg.ch_names)})
        data['target'].append(eeg.event_id[event_id])

baseline_features = pd.DataFrame(means).join(pd.DataFrame(stds))
baseline_features['target'] = data['target']
baseline_features.to_csv(path/'A4SHD_21F_inhale_features/baseline.csv')

In [34]:
columns = ['time'] + eeg.info['ch_names']
out_path = path / f'{subj}_inhale_csv'

path_file = []

if not out_path.exists():
    out_path.mkdir()

for event_id in eeg.event_id:
    for i in range(len(eeg[event_id])):
        f_name = f'{event_id}_{i}.csv'
        path_file.append({'fn': f_name,
                          'target': eeg.event_id[event_id],
                          'n_channels': eeg.info['nchan']})
        epoch = eeg[event_id][i].to_data_frame()[columns].set_index('time')
        epoch.columns = epoch.columns.str.lower()
        epoch.to_csv(out_path / f_name)
path_file = pd.DataFrame(path_file)
path_file['seconds'] = 7.0
path_file.to_csv(out_path / 'path_file.csv')

FileExistsError: [Errno 17] File exists: '../data/smell/99CEM_22F_inhale_csv'

In [ ]:
!python3 ../depr_experiments/191104_splits_calc_feats.py

Started features stage - coh-alpha
100%|███████████████████████████████████████████| 58/58 [00:04<00:00, 13.19it/s]
Started features stage - coh-beta
100%|███████████████████████████████████████████| 58/58 [00:03<00:00, 15.82it/s]
Started features stage - coh-theta
100%|███████████████████████████████████████████| 58/58 [00:03<00:00, 15.40it/s]
Started features stage - env-alpha
100%|███████████████████████████████████████████| 58/58 [00:01<00:00, 43.19it/s]
Started features stage - env-beta
100%|███████████████████████████████████████████| 58/58 [00:01<00:00, 43.75it/s]
Started features stage - env-theta
100%|███████████████████████████████████████████| 58/58 [00:01<00:00, 42.97it/s]
Started features stage - bands
  0%|                                                    | 0/58 [00:00<?, ?it/s]/home/max/anaconda3/envs/neuroml/lib/python3.9/site-packages/scipy/signal/spectral.py:1961: UserWarning: nperseg = 1024 is greater than input length  = 875, using nperseg = 875
  warnings.warn('n